# Phase 3v2: QLoRA 파인튜닝 (SOS/EOS + CoT 학습)

## 📌 목적
Phase 2v2에서 생성한 `training_dataset_v2.jsonl`을 사용하여
Unsloth 기반 QLoRA 파인튜닝을 수행합니다.

### 기존 대비 개선점
- ✅ SOS/EOS 토큰 포함 → 생성 종료 시점 학습
- ✅ CoT(Thinking) 포함 → 추론 과정 학습
- ✅ 노이즈 제거된 고품질 데이터
- ✅ RPM 기반 피처/팩터 정보 포함

## ⚙️ 사전 조건
- `phase2_dataset_v2.ipynb` 실행으로 `outputs/training_dataset_v2.jsonl` 존재
- Colab T4 이상 GPU 할당

## 📂 출력 파일
```
models/
├── llama3_popular_post_lora_v2/          # LoRA 어댑터
└── llama3_popular_post_lora_v2/merged_16bit/  # 16bit 병합 모델
```

---
## 0. 필수 패키지 설치

In [1]:
# Colab에는 이미 대부분의 패키지가 설치되어 있으므로 최소 설치
%pip install -q unsloth
%pip install -q datasets

print("✅ 패키지 설치 완료")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 116.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

print("✅ Google Drive 마운트 완료")

Mounted at /content/drive
✅ Google Drive 마운트 완료


---
## 1. 경로 및 데이터 로드

In [3]:
from pathlib import Path
import json
from datasets import Dataset

PROJECT_ROOT = Path("/content/drive/MyDrive/board_crawling")
OUTPUT_DIR   = PROJECT_ROOT / "outputs"
MODEL_DIR    = PROJECT_ROOT / "models"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_JSONL = OUTPUT_DIR / "training_dataset_v2.jsonl"
if not TRAIN_JSONL.exists():
    raise FileNotFoundError(f"{TRAIN_JSONL}가 없습니다. Phase 2v2를 먼저 실행하세요.")

training_data = []
with open(TRAIN_JSONL, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            training_data.append(json.loads(line))

print(f"✅ 데이터셋 로드 완료: {len(training_data):,}개 레코드")

✅ 데이터셋 로드 완료: 1,998개 레코드


---
## 2. Unsloth 모델 로드 + QLoRA 설정

In [4]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print(f"✅ Unsloth 모델 로드 완료")
print(f"  BOS token: {tokenizer.bos_token} (id: {tokenizer.bos_token_id})")
print(f"  EOS token: {tokenizer.eos_token} (id: {tokenizer.eos_token_id})")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.494 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Unsloth 2026.2.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


✅ Unsloth 모델 로드 완료
  BOS token: <|begin_of_text|> (id: 128000)
  EOS token: <|end_of_text|> (id: 128001)


---
## 3. 데이터셋 템플릿 적용 (SOS/EOS 토큰 확인)

In [5]:
# ── text 필드의 SOS/EOS를 실제 토크나이저 토큰으로 대체 ──────
BOS = tokenizer.bos_token  # <|begin_of_text|>
EOS = tokenizer.eos_token  # <|end_of_text|>

def fix_special_tokens(text: str) -> str:
    """placeholder를 실제 토크나이저 토큰으로 대체.
    Llama-3에서는 placeholder와 실제 토큰이 동일하므로
    이 함수는 안전장치 역할을 합니다.
    다른 모델 사용 시 여기서 매핑을 변경하세요."""
    text = text.replace("<|begin_of_text|>", BOS)
    text = text.replace("<|end_of_text|>", EOS)
    return text

for record in training_data:
    if "text" in record:
        record["text"] = fix_special_tokens(record["text"])

dataset = Dataset.from_list(training_data)

# 검증: 첫 번째 샘플 확인
print("✅ 데이터셋 준비 완료")
print(f"   레코드 수: {len(dataset):,}")
print(f"\n📝 첫 번째 샘플 (앞 300자):")
print(dataset[0]["text"][:300])
print(f"\n📝 마지막 50자:")
print(dataset[0]["text"][-50:])

# 토큰화 검증: BOS/EOS가 올바르게 포함되는지 확인
sample_ids = tokenizer(dataset[0]["text"], return_tensors="pt").input_ids[0]
print(f"\n🔍 토큰화 검증:")
print(f"  첫 토큰 ID: {sample_ids[0].item()} (BOS={tokenizer.bos_token_id})")
print(f"  끝 토큰 ID: {sample_ids[-1].item()} (EOS={tokenizer.eos_token_id})")
print(f"  BOS 포함: {sample_ids[0].item() == tokenizer.bos_token_id}")
print(f"  EOS 포함: {sample_ids[-1].item() == tokenizer.eos_token_id}")

✅ 데이터셋 준비 완료
   레코드 수: 1,998

📝 첫 번째 샘플 (앞 300자):
<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
주어진 키워드와 인기요인 정보를 사용하여 연애상담소의 인기글 스타일로 글을 작성해줘.

### Input:
키워드: 남친은, 근데, 잘생겼는데, 나는, 객관적으로, 잘생겼어, 훈훈, 아닌
인기요인: 일반

### Thinking:
연애 과정에서 느끼는 외모에 대한 자존감 문제와 개인적

📝 마지막 50자:
예쁘다 하는 건가 생각 들고미치겠어....자존감이 낮은 거겠지?<|end_of_text|>

🔍 토큰화 검증:
  첫 토큰 ID: 128000 (BOS=128000)
  끝 토큰 ID: 128001 (EOS=128001)
  BOS 포함: True
  EOS 포함: True


---
## 4. SFTTrainer 설정 및 학습

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments

# ── 패딩 설정 (EOS 학습 보장) ─────────────────────────────
tokenizer.padding_side = "right"

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir=str(OUTPUT_DIR / "finetune_logs_v2"),
        save_strategy="epoch",
        save_total_limit=3,
    ),
)

print("✅ Trainer 설정 완료. 학습을 시작합니다...")
trainer.train()
print("✅ 학습 완료")

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/1998 [00:00<?, ? examples/s]

✅ Trainer 설정 완료. 학습을 시작합니다...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,998 | Num Epochs = 3 | Total steps = 750
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

wandb: Enter your choice:

 2


...

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tony34036 (tony34036-sogang-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
1,3.218400
2,3.166400
3,3.099200
4,3.084400
5,2.827400
6,2.592800
7,2.349000
8,2.277200
9,2.402000
10,2.166700


train/epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇███
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/grad_norm,▃▃▂▂▄▃▁▄▃▄▄▂▄▄▂▃▄▄▄▃▄▃▄▃▃▅▄▃▆▇▇▅▇▆▆██▇▆▇
train/learning_rate,▄████▇▇▇▆▆▆▆▆▆▆▅▄▄▄▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
train/loss,█▆▅▅▄▄▃▃▄▄▄▃▃▃▃▃▃▄▂▃▃▃▂▂▃▂▄▂▃▃▃▁▂▂▁▂▃▁▁▃
total_flos,1.076597809705943e+17
train/epoch,3
train/global_step,750
train/grad_norm,1.26679
train/learning_rate,0.0
train/loss,0.7826


✅ 학습 완료


---
## 5. 모델 저장 (LoRA + 16bit 병합)

In [9]:
import shutil
from pathlib import Path

OUTPUT_MODEL_DIR = MODEL_DIR / "llama3_popular_post_lora_v2"
OUTPUT_MODEL_DIR.mkdir(parents=True, exist_ok=True)

# 1. LoRA 어댑터 저장
model.save_pretrained(str(OUTPUT_MODEL_DIR))
tokenizer.save_pretrained(str(OUTPUT_MODEL_DIR))
print(f"✅ LoRA 어댑터 저장 완료: {OUTPUT_MODEL_DIR}")

# 2. 16bit 병합 모델 저장 (로컬에 먼저 저장 후 이동)
LOCAL_MERGED_DIR = Path("/content/temp_merged_model")
FINAL_MERGED_DIR = OUTPUT_MODEL_DIR / "merged_16bit"

print("🔄 모델 병합 시작 (로컬 디렉토리)...")
FastLanguageModel.for_inference(model)

try:
    model.save_pretrained_merged(
        str(LOCAL_MERGED_DIR),
        tokenizer,
        save_method="merged_16bit",
    )
    print(f"✅ 로컬 병합 완료. 드라이브로 복사 중...")

    # 드라이브 폴더 생성 및 복사
    if FINAL_MERGED_DIR.exists():
        shutil.rmtree(FINAL_MERGED_DIR)
    shutil.copytree(LOCAL_MERGED_DIR, FINAL_MERGED_DIR)

    print(f"✅ 16bit 병합 모델 저장 완료: {FINAL_MERGED_DIR}")
except Exception as e:
    print(f"❌ 병합 중 오류 발생: {e}")
    print("팁: GPU 메모리가 부족하거나 드라이브 용량이 부족할 수 있습니다. 또는 base model을 다시 로드해야 할 수 있습니다.")

✅ LoRA 어댑터 저장 완료: /content/drive/MyDrive/board_crawling/models/llama3_popular_post_lora_v2
🔄 모델 병합 시작 (로컬 디렉토리)...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/4 [00:00<?, ?it/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [00:57<00:00, 14.43s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [01:09<00:00, 17.48s/it]


Unsloth: Merge process complete. Saved to `/content/temp_merged_model`
✅ 로컬 병합 완료. 드라이브로 복사 중...
✅ 16bit 병합 모델 저장 완료: /content/drive/MyDrive/board_crawling/models/llama3_popular_post_lora_v2/merged_16bit


---
## 6. 다음 단계
→ `phase4_generation_v2.ipynb`에서 CoT 기반 인기글 생성 수행